In [1]:
import pandas as pd
import sqlalchemy
#from sqlalchemy import create_engine
from sqlalchemy.engine import create_engine


In [2]:
purchase_events_df = spark.read.parquet("/tmp/purchase_events")

In [3]:
purchase_events_df.show(10,truncate=False)

+-----------------------+---------+---------------------+---------------+---------+--------+-----------+-----+
|event_ts               |userid   |Host                 |event_type     |name     |strength|n_purchased|price|
+-----------------------+---------+---------------------+---------------+---------+--------+-----------+-----+
|2021-12-10 01:40:12.744|user-005 |user-005.comcast.com |purchase_knife |kukri    |500     |7          |1000 |
|2021-12-10 01:40:18.733|user-005 |user-005.comcast.com |purchase_knife |kukri    |500     |6          |1000 |
|2021-12-10 01:40:30.717|user-009 |user-009.comcast.com |purchase_shield|parma    |800     |4          |1500 |
|2021-12-10 01:40:50.152|user-003 |user-003.comcast.com |purchase_knife |kukri    |500     |1          |1000 |
|2021-12-10 01:40:53.178|user-007 |user-007.comcast.com |purchase_sword |excalibur|1000    |6          |2000 |
|2021-12-10 02:56:35.759|user-0010|user-0010.comcast.com|purchase_shield|parma    |800     |1          |1500 |
|

In [4]:
guid_events_df = spark.read.parquet("/tmp/guild_events")

In [6]:
guid_events_df.show(10,truncate=False)

+-----------------------+-----------+---------------------+----------+----------------------------+--------+-----------+-----+
|event_ts               |userid     |Host                 |event_type|name                        |strength|n_purchased|price|
+-----------------------+-----------+---------------------+----------+----------------------------+--------+-----------+-----+
|2021-12-09 14:12:15.144|'user-0010'|user-0010.comcast.com|join_guild|'Castle_of_Rock'            |1200    |1          |1000 |
|2021-12-09 14:12:16.55 |'user-003' |user-003.comcast.com |join_guild|'Game_of_Thrones'           |1500    |1          |2000 |
|2021-12-09 14:12:14.683|'user-0010'|user-0010.comcast.com|join_guild|'Knights_of_the_Round_Table'|5000    |1          |3000 |
|2021-12-09 14:12:14.676|'user-0010'|user-0010.comcast.com|join_guild|'Knights_of_the_Round_Table'|5000    |1          |3000 |
|2021-12-09 14:12:14.687|'user-0010'|user-0010.comcast.com|join_guild|'Knights_of_the_Round_Table'|5000    |1  

In [5]:
fight_events_df = spark.read.parquet("/tmp/fight_events")

In [6]:
fight_events_df.show(10,truncate=False)

+-----------------------+---------+---------------------+-----------+-----+----------+
|event_ts               |userid   |Host                 |event_type |score|win_status|
+-----------------------+---------+---------------------+-----------+-----+----------+
|2021-12-10 02:56:46.434|user-0010|user-0010.comcast.com|fight_event|-10  |lost      |
|2021-12-10 02:56:52.472|user-0010|user-0010.comcast.com|fight_event|-10  |lost      |
|2021-12-10 01:40:21.888|user-005 |user-005.comcast.com |fight_event|100  |won       |
|2021-12-10 01:40:31.633|user-0010|user-0010.comcast.com|fight_event|100  |won       |
|2021-12-10 02:20:50.532|user-006 |user-006.comcast.com |fight_event|-10  |lost      |
|2021-12-10 02:20:52.398|user-007 |user-007.comcast.com |fight_event|-10  |lost      |
|2021-12-10 02:56:58.123|user-005 |user-005.comcast.com |fight_event|100  |won       |
|2021-12-10 02:57:04.733|user-006 |user-006.comcast.com |fight_event|100  |won       |
|2021-12-10 01:41:52.614|user-006 |localhos

In [7]:
purchase_events_df.registerTempTable('purchase_events')

In [11]:
#Q1. Which item is most popular?
spark.sql("""
            select event_type,sum(n_purchased) as count_of_purchase
            from purchase_events
            group by event_type order  by count_of_purchase DESC 
          """).show(truncate=False)

+---------------+-----------------+
|event_type     |count_of_purchase|
+---------------+-----------------+
|purchase_shield|1679             |
|purchase_sword |1660             |
|purchase_knife |1519             |
+---------------+-----------------+



In [8]:
#Q2. In average, how many items a player purchases?
spark.sql("""select round(AVG(A.number_of_items)) as average_item from(
            select userid,sum(n_purchased) as number_of_items
            from purchase_events
            group by userid ) A     
          """).show(truncate=False)

+------------+
|average_item|
+------------+
|486.0       |
+------------+



In [9]:
#Q3. What items users purchased first?
spark.sql(""" select A.event_type,count(*) from
            (select event_ts,userid ,event_type,
            RANK() OVER(PARTITION BY userid ORDER BY event_ts DESC) Rank
            from purchase_events) A where Rank=1 group by A.event_type
          """).show(truncate=False)

+---------------+--------+
|event_type     |count(1)|
+---------------+--------+
|purchase_sword |5       |
|purchase_knife |3       |
|purchase_shield|2       |
+---------------+--------+



In [10]:
#Q4 How many users purchased the game so far?
spark.sql(""" select count(distinct (userid)) from purchase_events
          """).show(truncate=False)

+----------------------+
|count(DISTINCT userid)|
+----------------------+
|10                    |
+----------------------+



In [12]:
#Q5. Who gets most items in total and how many of them?
spark.sql("""select userid,sum(n_purchased) as count_of_purchase
            from purchase_events
            group by userid order by count_of_purchase DESC 
          """).show(truncate=False)

+---------+-----------------+
|userid   |count_of_purchase|
+---------+-----------------+
|user-003 |591              |
|user-004 |584              |
|user-005 |563              |
|user-002 |551              |
|user-007 |492              |
|user-008 |491              |
|user-006 |461              |
|user-001 |438              |
|user-009 |354              |
|user-0010|333              |
+---------+-----------------+



In [13]:
#Q6. Who spent the most money in todal and how much is it?
spark.sql("""select userid,sum(price) as sum_of_purchase
            from purchase_events
            group by userid order by sum_of_purchase DESC 
          """).show(truncate=False)

+---------+---------------+
|userid   |sum_of_purchase|
+---------+---------------+
|user-003 |169500         |
|user-004 |152500         |
|user-007 |141000         |
|user-002 |139500         |
|user-005 |130000         |
|user-008 |120000         |
|user-009 |115500         |
|user-006 |105500         |
|user-0010|104500         |
|user-001 |104000         |
+---------+---------------+



In [8]:
spark.sql("""
            select *
            from purchase_events
            Limit 1
          """).show(truncate=False)

+-----------------------+----------+--------------------+---------------+-----+--------+-----------+-----+
|event_ts               |userid    |Host                |event_type     |name |strength|n_purchased|price|
+-----------------------+----------+--------------------+---------------+-----+--------+-----------+-----+
|2021-12-09 14:12:17.084|'user-001'|user-001.comcast.com|purchase_shield|parma|800     |10         |1500 |
+-----------------------+----------+--------------------+---------------+-----+--------+-----------+-----+



In [15]:
purchase_events_df.printSchema()

root
 |-- event_ts: timestamp (nullable = true)
 |-- userid: string (nullable = true)
 |-- Host: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- strength: long (nullable = true)
 |-- n_purchased: long (nullable = true)
 |-- price: long (nullable = true)



In [16]:
purchase_events_df.columns

['event_ts',
 'userid',
 'Host',
 'event_type',
 'name',
 'strength',
 'n_purchased',
 'price']

In [17]:
guild_events_df = spark.read.parquet("/tmp/guild_events")

In [18]:
guild_events_df.registerTempTable('guild_events')

In [19]:
guild_events_df.printSchema()

root
 |-- event_ts: timestamp (nullable = true)
 |-- userid: string (nullable = true)
 |-- Host: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- strength: long (nullable = true)
 |-- n_purchased: long (nullable = true)
 |-- price: long (nullable = true)



In [22]:
spark.catalog.listTables()

[Table(name='guild_events', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='purchase_events', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='guild_events', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='purchase_events', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [20]:
q1_query = """
               SELECT event_type,
                   sum(amount_purchased) as ttl_purchased
               FROM   purchase_events
               GROUP  BY event_type 
           """

In [21]:
spark.sql(q1_query).show(truncate=False)

AnalysisException: "cannot resolve '`amount_purchased`' given input columns: [price, strength, n_purchased, name, event_type, userid, Host, event_ts]; line 3 pos 23;\n'Aggregate [event_type#75], [event_type#75, 'sum('amount_purchased) AS ttl_purchased#157]\n+- SubqueryAlias purchase_events\n   +- Relation[event_ts#72,userid#73,Host#74,event_type#75,name#76,strength#77L,n_purchased#78L,price#79L] parquet\n"